# Einleitung zum Versuch "Dilution Correction"
Mit diesem Versuchspaket erhaltet ihr ein umfassendes Verständnis für die Notwendigkeit und Methodik der Dilution Correction in der atmosphärischen Messtechnik. Ihr lernt, wie man vom rohen, feuchten Sensorsignal zu einem normierten, trockenen Wert gelangt und welche Rolle dabei physikalische Gesetze und Korrekturverfahren spielen.

## Hintergrund und Motivation

Moderne Sensoren zur Messung von Gaskonzentrationen (z. B. CO₂) liefern häufig ein **feuchtes Sensorsignal**. Das bedeutet, dass neben der tatsächlichen Gaskonzentration auch der Einfluss von Wasserdampf im Messvolumen enthalten ist. 

Ein wichtiger Schritt hierbei ist die Umrechnung von Messwerten in **trockene Luft**, häufig ausgedrückt als $CO_2^{dry}$ oder **dry air mole fraction**. Diese Normierung ist entscheidend, da Wasserdampf den gemessenen Wert "verwässert" und somit den Stoffmengenanteil (Mole Fraction) des Zielgases verfälscht. 

In [ ]:
import polars as pl
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# Add the parent directory to the system path
PROJECT_PATH = os.path.abspath(os.path.join(".."))

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)
    
from utils.plot_dataframes import plot_column, plot_columns, plot_column_difference

In [ ]:
df_p = pl.read_parquet("../data/4_Dilution_Correction/Reference_Picarro_G2401.parquet")
df_s = pl.read_parquet("../data/4_Dilution_Correction/Sensor_GMP343_Measurement.parquet")

In [ ]:
plot_columns(df_s, "datetime", ["CO2_out_of_sensor", "CO2_dry"])
plot_column_difference(df_s, "datetime", ["CO2_out_of_sensor", "CO2_dry"])
plot_column(df_s, "datetime", "h2o_v%")

Fragen

- Wie groß sind die Unterschiede zwischen Messung von $CO_2$ in trockener und feuchter Luft?
- Wie war das Wetter zum Zeitpunkt der Messungen? (Nutze das Internet)

# Relative und Absolute Luftfeuchtigkeit

In dieser Übung lernt ihr den Unterschied zwischen absoluter und relativer Luftfeuchtigkeit kennen und berechnet, wie sich bei konstanter absoluter Luftfeuchtigkeit die relative Luftfeuchtigkeit bei unterschiedlichen Temperaturen verändert.

---

## Hintergrund

### Absolute Luftfeuchtigkeit
Die absolute Luftfeuchtigkeit gibt an, wie viel Wasserdampf (in Gramm) in einem Kubikmeter Luft enthalten ist (g/m³). Sie misst die Masse des Wasserdampfs in einem bestimmten Luftvolumen.

### Relative Luftfeuchtigkeit
Die relative Luftfeuchtigkeit (RH) ist das Verhältnis des aktuellen Dampfdrucks $e$ zum Sättigungsdampfdruck $e_s(T)$ bei einer bestimmten Temperatur, ausgedrückt in Prozent:

$$
RH = \frac{e}{e_s(T)} \times 100 \%
$$

---

## Zusammenhang zwischen absoluter und relativer Luftfeuchtigkeit

Um den tatsächlichen Dampfdruck $e$ aus der absoluten Luftfeuchtigkeit $AH$ (in g/m³) zu berechnen, kann folgende Beziehung herangezogen werden:

$$
e = \frac{AH \times (T + 273.15)}{216.7}
$$

Hierbei ist:
- $T$ die Temperatur in °C,
- $e$ der Dampfdruck in hPa.

Der Sättigungsdampfdruck $e_s$ wird häufig mit der Tetens-Formel approximiert:

$$
e_s(T) = 6.112 \times \exp\left(\frac{17.67 \times T}{T + 243.5}\right)
$$

---

## Aufgabenstellung

Gegeben sei eine konstante absolute Luftfeuchtigkeit, z. B. $AH = 10 \, \text{g/m}^3$. Berechnet in Python für einen Temperaturbereich (z. B. von 0 °C bis 40 °C) die relative Luftfeuchtigkeit. Dabei soll für jede Temperatur:

1. Der tatsächliche Dampfdruck $e$ aus der absoluten Luftfeuchtigkeit berechnet werden.
2. Der Sättigungsdampfdruck $e_s$ mithilfe der Tetens-Formel berechnet werden.
3. Die relative Luftfeuchtigkeit $RH$ als Verhältnis $\frac{e}{e_s}$ in Prozent bestimmt werden.

In [ ]:
# Wichtige Konstanten
T0 = 273.15  # T0: float = The 0 C temperature in K
TC = 647.096  # TC: float = The critical point temperature of water
PC = 22.064e6  # PC: float = The critical point pressure of water
P0 = 1013.25e2  # P0: float = Reference pressure at sea level

# 🧪 Praktische Übung 1

✅ Aufgabe:

- Implementiere die Tetens Formel in Python

## Nützliche Funktionen:

3 + (1/2) = 3.5

$\text{2}^3$ = 2**3

$\text{e}^1$ = np.exp(1)

In [ ]:
# Funktion zur Berechnung des Sättigungsdampfdrucks e_s (in hPa) mittels Tetens-Formel
def saturation_vapor_pressure(T):
    """
    Berechnet den Sättigungsdampfdruck von Wasser (in Pascal) bei gegebener Temperatur T in Kelvin
    mithilfe der Tetens-Formel.
    
    Parameter
    - T: Temperatur in Kelvin
    
    Rückgabewert
    - e_s: Sättigungsdampfdruck in Pascal
    """
    # Tetens-Formel

    return e_s 

In [ ]:
# Prüfe deine Implementierung
assert(saturation_vapor_pressure(300) == 35.34519666889136)

In [ ]:
# Konstante absolute Luftfeuchtigkeit (g/m³)
AH = 10.0

# Temperaturbereich in °C (z. B. 0°C bis 40°C)
temperatures = np.linspace(0, 40, 100) + T0

# Funktion zur Berechnung des tatsächlichen Dampfdrucks e (in hPa)
def actual_vapor_pressure(AH, T):
    return (AH * (T + T0)) / 216.7

# Berechnung von e, e_s und der relativen Luftfeuchtigkeit RH
e = actual_vapor_pressure(AH, temperatures)
e_s = saturation_vapor_pressure(temperatures)  
RH = (e / e_s) * 100  # relative Luftfeuchtigkeit in Prozent

# Ausgabe einiger Ergebnisse
for T, rh in zip(temperatures[::10], RH[::10]):  # alle 10. Werte ausgeben
    print(f"Temperatur: {T:.1f} °C, Relative Luftfeuchtigkeit: {rh:.1f} %")

# Plotten der Ergebnisse
plt.figure(figsize=(8, 5))
plt.plot(temperatures - T0, RH, label='Relative Luftfeuchtigkeit')
plt.xlabel('Temperatur (°C)')
plt.ylabel('Relative Luftfeuchtigkeit (%)')
plt.title('Relative Luftfeuchtigkeit bei konstanter absoluter Luftfeuchtigkeit')
plt.grid(True)
plt.legend()
plt.show()

Fragen:

- Was bedeutet es, wenn die Luftfeuchtigkeit über 100% ist?

## Alternative Methode: Wagner-Gleichung zur Berechnung des Sättigungsdampfdrucks

Anstelle der Tetens-Formel kann zur Berechnung des Sättigungsdampfdrucks von Wasser auch die Wagner-Gleichung verwendet werden. Diese Methode bietet eine präzisere Näherung, insbesondere über einen breiteren Temperaturbereich.

### Formulierung der Wagner-Gleichung

Die Wagner-Gleichung lautet:

$$
\ln\left(\frac{P}{P_C}\right) = \frac{T_C}{T} \left( c_1 \theta + c_2 \theta^{1.5} + c_3 \theta^3 + c_4 \theta^{3.5} + c_5 \theta^4 + c_6 \theta^{7.5} \right)
$$

wobei

$$
\theta = 1 - \frac{T}{T_C}
$$

und die Parameter definiert sind als:

- $T_C$: Kritische Temperatur von Wasser (z. B. $T_C = 647.096\,\text{K}$)
- $P_C$: Kritischer Druck von Wasser (z. B. $P_C = 22064000\,\text{Pa}$)
- $c_1 = -7.85951783$
- $c_2 = 1.84408259$
- $c_3 = -11.7866497$
- $c_4 = 22.6807411$
- $c_5 = -15.9618719$
- $c_6 = 1.80122502$

Mit dieser Gleichung kann der Sättigungsdampfdruck $P$ (auch $e_s$ genannt) bei einer gegebenen Temperatur $T$ in Kelvin berechnet werden.

# 🧪 Praktische Übung 2

✅ Aufgabe:

- Implementiere die Wagner Gleichung in Python

In [ ]:
# Koeffizienten der Wagner-Gleichung
c1 = -7.85951783
c2 = 1.84408259
c3 = -11.7866497
c4 = 22.6807411
c5 = -15.9618719
c6 = 1.80122502

def saturation_vapor_pressure_wagner(T):
    """
    Berechnet den Sättigungsdampfdruck von Wasser (in Pascal) bei gegebener Temperatur T in Kelvin
    mithilfe der Wagner-Gleichung.
    
    Parameter:
    - T: Temperatur in Kelvin
    
    Rückgabe:
    - P: Sättigungsdampfdruck in Pascal
    """
    # Berechnung Theta
    theta = 1 - T / TC
    
    # Berechnung ln(P/P_C): x 
    x = TC / T * (
                c1 * theta
                + c2 * theta**1.5
                + c3 * theta**3
                + c4 * theta**3.5
                + c5 * theta**4
                + c6 * theta**7.5
            )
    
    # Berechne den Sättigungsdampfdrucks P (Tipp: Verwende die Exponentialfunktion np.exp())
    
    return P

In [ ]:
assert(saturation_vapor_pressure_wagner(300) == 3536.7175865049244)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Konstante absolute Luftfeuchtigkeit (g/m³)
AH = 10.0

# Temperaturbereich in °C (z. B. 0°C bis 40°C)
temperatures = np.linspace(0, 40, 100) + T0 # Umwandlung in Kelvin

# Funktion zur Berechnung des tatsächlichen Dampfdrucks e (in Pa)
def actual_vapor_pressure(AH, T):
    return (AH * (T)) / 216.7 * 100

# Berechnung von e, e_s und der relativen Luftfeuchtigkeit RH
e = actual_vapor_pressure(AH, temperatures)
e_s = saturation_vapor_pressure_wagner(temperatures)  # Umwandlung in Kelvin für die Wagner-Gleichung
RH = (e / e_s) * 100  # relative Luftfeuchtigkeit in Prozent

# Ausgabe einiger Ergebnisse
for T, rh in zip(temperatures[::10], RH[::10]):  # alle 10. Werte ausgeben
    print(f"Temperatur: {T:.1f} °C, Relative Luftfeuchtigkeit: {rh:.1f} %")

# Plotten der Ergebnisse
plt.figure(figsize=(8, 5))
plt.plot(temperatures - T0, RH, label='Relative Luftfeuchtigkeit')
plt.xlabel('Temperatur (°C)')
plt.ylabel('Relative Luftfeuchtigkeit (%)')
plt.title('Relative Luftfeuchtigkeit bei konstanter absoluter Luftfeuchtigkeit')
plt.grid(True)
plt.legend()
plt.show()

Fragen

- Wie unterscheiden sich die Ergebnisse von den Annäherungen durch Tetens und Wagner?
- Was bedeutet es für die absolute Luftfeuchtigkeit, wenn man die gleiche relative Luftfeuchtigkeit bei unterschiedlichen Temperaturen misst?

# Kurze Erklärung des Taupunkts

Der **Taupunkt** ist die Temperatur, bei der die in der Luft enthaltene Wasserdampfmenge so hoch ist, dass die Luft bei weiterer Abkühlung gesättigt wird (d.h. eine relative Luftfeuchtigkeit von 100 % erreicht). Bei Erreichen des Taupunkts beginnt der Wasserdampf zu kondensieren, was zur Bildung von Tau, Nebel oder Wolken führen kann. Der Taupunkt ist daher ein wichtiges Maß, um die Feuchtigkeitsverhältnisse in der Atmosphäre zu beschreiben.

- **Tetens-Formel:**  
  Die Tetens-Formel ist relativ einfach und wird in der Praxis häufig verwendet. Sie eignet sich besonders für schnelle Abschätzungen.

- **Wagner-Gleichung:**  
  Die Wagner-Gleichung bietet eine deutlich höhere Genauigkeit. Sie wird vor allem in wissenschaftlichen Anwendungen genutzt, bei denen exakte thermodynamische Eigenschaften von Wasser von Bedeutung sind.

---

# Partialdruck und Dalton's Law

In dieser Übung werdet ihr den Zusammenhang zwischen Partialdruck, Gesamtdruck und den Stoffmengenanteil (mole fraction) in Gasgemischen verstehen und anwenden. Dabei spielt das **Daltonsche Gesetz** eine zentrale Rolle.

---

## Hintergrund

### Partialdruck
Der **Partialdruck** eines Gases in einem Gemisch ist der Druck, den dieses Gas ausüben würde, wenn es allein im Behälter vorhanden wäre. Für ein Gas $i$ gilt:

$$
P_i = x_i \times P_{\text{total}}
$$

wobei:
- $P_i$ der Partialdruck des Gases $i$ ist,
- $x_i$ der Stoffmengeanteil (Mole Fraction) von $i$ ist,
- $P_{\text{total}}$ der Gesamtdruck des Gasgemisches ist.

### Dalton'sches Gesetz
Das **Dalton'sche Gesetz** besagt, dass der Gesamtdruck eines Gemisches gleich der Summe der Partialdrücke der einzelnen Gase ist:

$$
P_{\text{total}} = \sum_{i} P_i
$$

Diese Beziehung ermöglicht es, die Partialdrücke einzelner Komponenten zu berechnen und zu überprüfen, ob ihre Summe dem Gesamtdruck entspricht.

---



# 🧪 Praktische Übung 3

1. **Theoretische Grundlagen:**  
   Lest die obigen Erklärungen und stellt sicher, dass ihr den Zusammenhang zwischen den Größen versteht.

2. **Berechnung der Partialdrücke:**  
   Gegeben ist ein Gasgemisch mit einem Gesamtdruck $P_{\text{total}}$ und den Stoffmengenanteil der einzelnen Gase.  
   Beispiel:  
   - $P_{\text{total}} = 101325 \, \text{Pa}$ (1 atm)  
   - Luftzusammensetzung (vereinfachtes Modell):
     - $x_{\text{N}_2} = 0.78$
     - $x_{\text{O}_2} = 0.21$
     - $x_{\text{CO}_2} = 0.01$



✅ Aufgabe:

-   Berechnet für jedes Gas den Partialdruck $P_i$ und überprüft, ob gilt:

$$P_{\text{total}} = P_{\text{N}_2} + P_{\text{O}_2} + P_{\text{CO}_2}$$

In [ ]:
# Nützliche Funktionen

dictionary = {}
dictionary["a"] = 1
dictionary["b"] = 2

print(dictionary)

for key, value in dictionary.items():
    print(key, value)

In [ ]:
def calculate_partial_pressures(gas_mole_fractions, total_pressure):
    """
    Berechnet die Partialdrücke einzelner Gase in einem Gemisch.
    
    Parameter:
    - gas_mole_fractions: dict, wobei die Keys die Gasnamen und die Werte die Stoffmengenanteile sind.
    - total_pressure: float, Gesamtdruck des Gasgemisches in Pascal.
    
    Rückgabe:
    - dict: Partialdruck der einzelnen Gase in Pascal.
    """
    partial_pressures = {}
    
    # Berechnung der Partialdrücke
    for gas, mole_fraction in gas_mole_fractions.items():
          
        
    return partial_pressures

In [ ]:
assert(calculate_partial_pressures({"Test": 0.33}, 101325) == {"Test": 33437.25})

In [ ]:
# Beispiel: Berechnung der Partialdrücke (CO₂ ≈ 420 ppm)

#In der Atmosphäre liegt der CO₂-Gehalt typischerweise bei etwa 420 ppm, was einer Stoffmengenanteil (mole fraction) von 420 (1/1_000_000) = 0.00042 entspricht.  

# Beispiel-Daten
total_pressure = 101325  # Gesamtdruck in Pascal
gas_mole_fractions = {
    "N2": 0.78,
    "O2": 0.21958,
    "CO2": 0.00042  # entspricht 420 ppm
}

# Berechnung der Partialdrücke
partial_pressures = calculate_partial_pressures(gas_mole_fractions, total_pressure)
print("Berechnete Partialdrücke (in Pa):")
for gas, pressure in partial_pressures.items():
    print(f"{gas}: {pressure:.2f} Pa")

# Überprüfung des Dalton'schen Gesetzes
sum_partial_pressures = sum(partial_pressures.values())
print(f"\nSumme der Partialdrücke: {sum_partial_pressures:.2f} Pa")
print(f"Gesamtdruck: {total_pressure:.2f} Pa")

---

In der Realität ist Luft ein Gemisch aus trockenen Gasen (wie $N_2$, $O_2$, $CO_2$ usw.) und Wasserdampf. Der Anteil an Wasserdampf ist variabel und wird oft durch die relative Luftfeuchtigkeit angegeben. Dieser Wasserdampf hat seinen eigenen Partialdruck, der gemäß Dalton'schem Gesetz in die Summe aller Partialdrücke eingeht:

$$
P_{\text{total}} = P_{\text{dry air}} + P_{\text{H}_2\text{O}}
$$

Das bedeutet:
- **Wenn der Anteil an Wasserdampf steigt:**  
  Erhöht sich der Partialdruck von Wasser ($P_{\text{H}_2\text{O}}$). Bei konstantem Gesamtdruck muss der Partialdruck der trockenen Luftkomponenten entsprechend sinken, da ihre Summe zusammen mit dem Wasserdampfdruck den Gesamtdruck ergibt.
  
- **Auswirkung auf die Messung:**  
  Wird der Wasserdampfanteil nicht berücksichtigt, kann es zu Fehlern kommen, wenn man z. B. Gaskonzentrationen (wie CO₂) bezogen auf trockene Luft angibt. Deshalb rechnet man oft von "wet" (feuchter) zu "dry" (trockener) Luft, um vergleichbare Werte zu erhalten.

Kurz zusammengefasst:  
Der variable Wasserdampfanteil verändert die Partialdrücke der anderen Gase, da der Gesamtdruck immer die Summe der Partialdrücke aller enthaltenen Gase ist. Ein hoher Wasserdampfanteil führt also zu einem höheren $P_{\text{H}_2\text{O}}$ und entsprechend zu geringeren Partialdrücken der trockenen Komponenten, wenn der Gesamtdruck konstant bleibt.

---

# Umrechnung der relativen Luftfeuchtigkeit in das Stoffmengenverhältnis

In dieser Übung wandeln wir die relative Luftfeuchtigkeit $rh$ (in %) in das Stoffmengenverhältnis $x_{\text{H}_2\text{O}}$ um. Wir nehmen dabeian, dass die Funktion zur Berechnung des Sättigungsdampfdrucks $e_s(T)$ durch die Wagner-Gleichung bestimmt wird.

---

## Hintergrund

$$
P_i = x_i \times P_{\text{total}}
$$

$$
rh = \frac{e}{e_s} \times 100
$$

Die relative Luftfeuchtigkeit (rH) gibt an, wie viel Prozent des maximal möglichen Wasserdampfdrucks bei einer bestimmten Temperatur vorhanden sind. Der maximale, sätigungsbezogene Dampfdruck bei einer Temperatur $T$ wird durch $e_s(T)$ beschrieben. Zur Umrechnung der relativen Luftfeuchtigkeit in den tatsächlichen Dampfdruck gilt:


$$
P_{\text{H}_2\text{O}} =  e_s(T) \times \frac{rh}{100} 
$$

Der Sättigungsdampfdruck $e_s(T)$ bei Temperatur $T$ (in Kelvin) wird hier über die Wagner-Gleichung berechnet (Details sind euch bereits bekannt). Zur Umrechnung der relativen Luftfeuchtigkeit in das Stoffmengenverhältnis nutzen wir die folgende Formel:

$$
x_{\text{H}_2\text{O}} =   \frac{P_{\text{H}_2\text{O}}}{P_{\text{total}}} =  \frac{e_s(T) \times \frac{rh}{100}}{P_{\text{total}}}
$$

wobei:
- $x_{\text{H}_2\text{O}}$ das Stoffmengenverhältnis ist,
- $e_s(T)$ der Sättigungsdampfdruck (in Pascal) bei der Temperatur $T$ ist,
- $rh$ die relative Luftfeuchtigkeit (in %) und
- $p$ der Umgebungsdruck (in Pascal).

---

# 🧪 Praktische Übung 4

**Verständnisfragen:**
   - Erklärt in eigenen Worten, was das Stoffmengenverhältnis $x_{\text{H}_2\text{O}}$ beschreibt.
   - Wie wirkt sich eine Änderung der relativen Luftfeuchtigkeit $rh$ oder des Umgebungsdrucks $p$ auf das Stoffmengenverhältnis aus?

✅ Aufgabe:

   - Nutzt die gegebene Umrechnungsformel, um bei vorgegebenen Werten von $rh$, $T$ und $p$ das Stoffmengenverhältnis $x_{\text{H}_2\text{O}}$ zu berechnen.
   - Beispielwerte:  
     - $rh = 40\%$,  
     - $T =  20°C$,  
     - $p = 101325 \, \text{Pa}$.

In [ ]:
# Wir nehmen an, dass die Funktion saturation_vapor_pressure(T)
# den Sättigungsdampfdruck e_s(T) in Pa mittels der Wagner-Gleichung berechnet.
# Die Implementierung dieser Funktion ist euch bekannt.

def rh_to_mole_fraction(rh, T, p):
    """
    Konvertiert die relative Luftfeuchtigkeit (rh in %) in das Stoffmengenverhältnis xh2o.

    Parameter:
    - rh: relative Luftfeuchtigkeit in %
    - T: absolute Temperatur in K
    - p: Umgebungsdruck in Pa

    Rückgabe:
    - x_h2o: Molar mixing ratio
    """
    # Umrechnung von rh in Molar Mixing Ratio
    
    
    return x_h2o

In [ ]:
assert(rh_to_mole_fraction(50, 300, 101325) == 0.017452344369627063)

In [ ]:
# Beispiel: rH = 40%, T = 20 °C, p = 101325 Pa

rh_example = 40
T_example = 20 + T0  # Umwandlung in Kelvin
p_example = 101325  # in Pascal

x_h2o = rh_to_mole_fraction(rh_example, T_example, p_example)
print(f"Stoffmengenverhältnis: {x_h2o:.6f}")

Fragen

- Verändere die Parameter: Wie hoch kann das Stoffmengenverhältnis (mole fraction) von Wasserdampf maximal in der Atmosphäre unter typischen Bedingungen sein?
- Wie verändern sich die Partialdrücke bzw. Mole Fractions der anderen Gase, wenn in einem bestimmten Volumen mehr Wasserdampf vorhanden ist?

---

# Umrechnung von $CO_2^{wet}$ zu $CO_2^{dry}$ mittels des Stoffmengenanteils von Wasser

In der atmosphärischen Messtechnik wird die CO₂-Konzentration häufig als $CO_2^{wet}$ (in feuchter Luft) gemessen. Da in der feuchten Luft auch Wasserdampf enthalten ist, entspricht der gemessene Wert nicht dem tatsächlichen CO₂-Stoffmengenanteil der trockenen Luft. Um vergleichbare Daten zu erhalten, wird der Einfluss des Wasserdampfs korrigiert, so erhalten wir den $CO_2^{dry}$, also den CO₂-Stoffmengenanteil bezogen auf die trockene Luft. In der Literatur wird $CO_2^{dry}$ häufig als "dry air mole fraction" bezeichnet.


## Hintergrund

Dies erfolgt mit folgender Gleichung:

$$
CO_2^{dry} = \frac{CO_2^{wet}}{1 - x_{H_2O}},
$$

wobei
- $CO_2^{wet}$ die CO₂-Konzentration in feuchter Luft (z.B. in ppm) ist,
- $x_{H_2O}$ der Stoffmengenanteil des Wasserdampfs in der Luft darstellt.

**Beispiel:**  
Angenommen, die gemessene CO₂-Konzentration in feuchter Luft beträgt 420 ppm und der Wasseranteil liegt bei $x_{H_2O} = 0.01$ (also 1 %). Dann berechnet sich:

$$
CO_2^{dry} = \frac{420 \, \text{ppm}}{1 - 0.01} = \frac{420}{0.99} \approx 424.24 \, \text{ppm}.
$$

Das bedeutet, dass die tatsächliche CO₂-Konzentration in der trockenen Luft höher ist, da der Anteil des Wasserdampfs den gemessenen Wert „verwässert“.

# 🧪 Praktische Übung 5

**Theoretische Fragen:**
   - Erklärt in eigenen Worten, warum es notwendig ist, von $CO_2^{wet}$ zu $CO_2^{dry}$ umzurechnen.
   - Diskutiert, wie sich ein Anstieg des Stoffmengenanteils von Wasser ($x_{H_2O}$) auf die berechnete $CO_2^{dry}$-Konzentration auswirkt.

✅ Aufgaben:

1. **Berechnungsaufgabe:**
   - Berechnet anhand der oben genannten Gleichung $CO_2^{dry}$, wenn $CO_2^{wet} = 420$ ppm und $x_{H_2O} = 0.01$.

2. **Programmierung in Python:**
  - Schreibt eine Funktion, die den $CO_2^{dry}$-Wert aus dem gemessenen $CO_2^{wet}$ und dem Stoffmengenanteil des Wasserdampfs $x_{H_2O}$ berechnet.
   - Testet eure Funktion mit verschiedenen Eingabewerten.


In [ ]:
def convert_co2_wet_to_dry(co2_wet, x_h2o):
    """
    Konvertiert den CO2-Stoffmengenanteil in feuchter Luft (co2_wet) in den
    Stoffmengenanteil in trockener Luft (co2_dry).

    Parameter:
    - co2_wet: CO2-Konzentration in feuchter Luft (z.B. in ppm)
    - x_h2o: Stoffmengenanteil des Wasserdampfs, z.B. 0.01 für 1%

    Rückgabe:
    - co2_dry: korrigierte CO2-Konzentration in trockener Luft (ppm)
    """
    # Berechnung des CO2-Anteils in trockener Luft
    
    return co2_dry

In [ ]:
assert(convert_co2_wet_to_dry(400, 0.01) == 404.04040404040404)

In [ ]:
# Beispiel: co2_wet = 420 ppm, x_h2o = 0.01 (1 %)
co2_wet = 420
x_h2o = 0.01
co2_dry = convert_co2_wet_to_dry(co2_wet, x_h2o)
print(f"CO2_dry: {co2_dry:.2f} ppm")

---

# Anwendung der Dilution Correction auf unsere Zeitreihe

In dieser Übung wenden wir die Dilution Correction auf eine Zeitreihe an, um den Einfluss des Wasserdampfs auf die CO₂-Messungen zu korrigieren. Hierfür nutzen wir die von euch zuvor implementieren Python Funktionen `rh_to_mole_fraction()` und `convert_co2_wet_to_dry()`.

Anschließend lassen wir uns das DataFrame mit den korrigierten Werten ausgeben. 

Die Dilution Correction ist meistens der erste Schritt in unserer Pipeline, noch bevor wir eine Korrektur durch trockenen Kalibriergase vornehmen. 

In [ ]:
def absolute_temperature(temperature: float) -> float:
    """
    Convert temperature in Celsius to Kelvin.
    
    Parameters:
    temperature (float): Temperature in Celsius.
    
    Returns:
    float: Temperature in Kelvin.
    """
    return temperature + T0

def wet_to_dry_mole_fraction(df_wet: pl.DataFrame) -> pl.DataFrame:
    # perform dry conversion for measurement data
    return df_wet.with_columns(pl.struct(['gmp343_temperature','sht45_humidity','bme280_pressure'])
        .map_elements(lambda x: (rh_to_mole_fraction(x['sht45_humidity'],absolute_temperature(x['gmp343_temperature']),x['bme280_pressure']*100)), return_dtype=pl.Float64) \
        .alias("x_h2o")) \
        .with_columns(pl.struct(['CO2_out_of_sensor','x_h2o']) \
        .map_elements(lambda x: convert_co2_wet_to_dry(x['CO2_out_of_sensor'],x['x_h2o']), return_dtype=pl.Float64)
        .alias("CO2_dry_calc"))

In [ ]:
df = df_s.select(["datetime","CO2_out_of_sensor","gmp343_temperature","sht45_humidity","bme280_pressure"])
df.head()

In [ ]:
df_dry = df.pipe(wet_to_dry_mole_fraction)
df_dry.head()

In [ ]:
df_plot = df_dry.join(df_s.select(["datetime", "CO2_dry"]), on="datetime", how="left")

plot_columns(df_plot, "datetime", ["CO2_dry_calc", "CO2_dry"])
plot_column_difference(df_plot, "datetime", ["CO2_dry_calc", "CO2_dry"])